In [1]:
import scipy.signal as sig
import numpy as np
import time

from open_ephys.analysis import Session
import pyopenephys as oe


In [2]:
DATA_FOLDER = "/data/jpatriota/R14/3.Probe test/2021-09-23_00-16-10"
EXP_FOLDER = "/data/jpatriota/R14/3.Probe test/2021-09-23_00-16-10/Record Node 107/"
session = Session(DATA_FOLDER)

info = session.recordnodes[0].recordings[0].info['continuous'][0]
Fs = info['sample_rate']
channels = info['channels']
channel_names = [c['channel_name'] for c in channels]
unique_channel_names = np.unique(channel_names)
unique_data_channel_names = np.unique(channel_names[0:127])
brain_regions = ["A1T", "BLAT", "HPCT", "PFCT"]

# dead_channels = {105, 106, 90, 91, 66, 67, 68, 69, 70, 36, 37, 38, 125, 126, 127, 122, 111, 97, 98, 101, 102, 84, 29, 30, 31}


# cannot load data via session (permission denied)
# samples = session.recordnodes[0].recordings[0].continuous[0].samples

data_file = oe.File(EXP_FOLDER)
experiments = data_file.experiments

experiment = experiments[0]
recordings = experiment.recordings

rec1 = recordings[0]

signal = rec1.analog_signals[0].signal
# ts = np.asarray(rec1.analog_signals[0].times)

Loading Open-Ephys: reading settings...
Decoding data from  binary  format
Reading oebin file
Reading oebin file


In [3]:
fs_target = 1000
ds_factor = Fs // fs_target
ds_length = signal.shape[1] // ds_factor

In [4]:
t0 = time.time()
downsampled_signal_loop = np.empty((signal.shape[0], ds_length))
# downsampled_ts = np.empty(ds_length)

for idx in range(ds_length):
    downsampled_signal_loop[:, idx] = signal[:, idx * ds_factor] 
    # downsampled_ts[idx] = ts[idx * ds_factor]

t1 = time.time()
print(f"Loop time: {t1 - t0}")

Loop time: 103.10846590995789


In [7]:
t0 = time.time()
ds_idx = np.arange(0, signal.shape[1]-1, ds_factor, dtype=int)
downsampled_signal_dec = signal[:, ds_idx]
t1 = time.time()
print(f"dec time: {t1 - t0}")



dec time: 109.44291973114014


In [8]:
print(downsampled_signal_loop)
print(downsampled_signal_dec)

[[-1.0400e+02  1.0800e+02  7.2000e+01 ... -6.8200e+02 -5.9900e+02
  -1.8500e+02]
 [-2.0000e+01  1.7500e+02  1.3700e+02 ... -8.6600e+02 -8.7600e+02
  -5.6300e+02]
 [-1.2300e+02  7.5000e+01  6.5000e+01 ... -9.0400e+02 -9.8300e+02
  -6.0400e+02]
 ...
 [ 2.7500e+02  2.5100e+02  2.3300e+02 ...  2.4300e+02  2.5900e+02
   2.5900e+02]
 [-2.8459e+04 -2.8471e+04 -2.8465e+04 ... -3.0455e+04 -3.0459e+04
  -3.0463e+04]
 [-2.6686e+04 -2.6697e+04 -2.6689e+04 ... -3.0539e+04 -3.0543e+04
  -3.0547e+04]]
[[  -104    108     72 ...   -599   -185   -465]
 [   -20    175    137 ...   -876   -563   -800]
 [  -123     75     65 ...   -983   -604   -750]
 ...
 [   275    251    233 ...    259    259    271]
 [-28459 -28471 -28465 ... -30459 -30463 -30453]
 [-26686 -26697 -26689 ... -30543 -30547 -30537]]
